# <span style="color:blue">MSc em Ciência de Dados</span>
# <span style="color:blue">Análise de Dados com Base em Processamento Massivo em Paralelo</span>

## <span style="color:blue">Avaliação Final - Notebook </span>

**Material Produzido por:**<br>
>**Profa. Dra. Cristina Dutra de Aguiar**<br>


**CEMEAI - ICMC/USP São Carlos**

Este *notebook* deve conter as respostas para as consultas analíticas solicitadas nas Questões 8, 9 e 10. É possível especificar as consultas analíticas usando Pandas, o método spark.sql() ou os métodos do módulo pyspark.sql. As consultas devem ser especificadas na seção 6, na célula indicada para resposta.

**IMPORTANTE**

- **As respostas para as Questões 8, 9 e 10 somente serão consideradas se forem especificadas no *notebook*. Portanto, independentemente da alternativa estar certa ou errada, se a consulta analítica correspondente não for especificada, a alternativa será considerada errada.**

- **Caso uma alternativa esteja certa, porém a especificação da consulta analítica correspondente estiver errada no *notebook*, a alternativa será considerada errada.**


O *notebook* contém a constelação de fatos da BI Solutions que deve ser utilizada para responder às questões e também todas as bibliotecas, bases de dados, inicializações, instalações, importações, geração de dataFrames, geração de visões temporárias e conversão dos tipos de dados necessárias para a realização da questão. Portanto, o *notebook* está preparado para ser executado usando Pandas, o método spark.sql() e os métodos do módulo pyspark.sql.

O uso do *framework* Spark requer diversas configurações no ambiente de desenvolvimento para executar o *notebook*. Dado que tal complexidade foge do escopo de nossa disciplina, recomenda-se que o *notebook* seja executado na plataforma de desenvolvimento COLAB. O uso do COLAB  proporciona um ambiente de desenvolvimento pré-configurado e remove a complexidade de instalação e configuração de pacotes e *frameworks* que são utilizados na disciplina.

**INSTRUÇÕES DE ENTREGA**

**O que deve ser entregue:**
- **O notebook com as respostas no formato .ipynb**
- **O notebook com as respostas no formato .pdf**

**Ambos arquivos devem ser nomeados usando o primeiro nome e o último sobrenome do aluno. Por exemplo: CristinaAguiar.ipynb e CristinaAguiar.pdf.**

Boa avaliação!

#1 Constelação de Fatos da BI Solutions

A aplicação de *data warehousing* da BI Solutions utiliza como base uma constelação de fatos, conforme descrita a seguir.

**Tabelas de dimensão**

- data (dataPK, dataCompleta, dataDia, dataMes, dataBimestre, dataTrimestre, dataSemestre, dataAno)
- funcionario (funcPK, funcMatricula, funcNome, funcSexo, funcDataNascimento, funcDiaNascimento, funcMesNascimento, funcAnoNascimento, funcCidade, funcEstadoNome, funcEstadoSigla, funcRegiaoNome, funcRegiaoSigla, funcPaisNome, funcPaisSigla)
- equipe (equipePK, equipeNome, filialNome, filialCidade, filialEstadoNome, filialEstadoSigla, filialRegiaoNome, filialRegiaoSigla, filialPaisNome, filialPaisSigla)
- cargo (cargoPK, cargoNome, cargoRegimeTrabalho, cargoEscolaridadeMinima, cargoNivel)
- cliente (clientePK, clienteNomeFantasia, clienteSetor, clienteCidade, clienteEstadoNome, clienteEstadoSigla, clienteRegiaoNome, clienteRegiaoSigla, clientePaisNome, clientePaisSigla)

**Tabelas de fatos**
- pagamento (dataPK, funcPK, equipePK, cargoPK, salario, quantidadeLancamentos)
- negociacao (dataPK, equipePK, clientePK, receita, quantidadeNegociacoes)


#2 Obtenção dos Dados da BI Solutions


## 2.1 Baixando o Módulo wget

Para baixar os dados referentes ao esquema relacional da constelação de fatos da BI Solutions, é utilizado o módulo  **wget**. O comando a seguir realiza a instalação desse módulo. <br>

In [ ]:
#instalando o módulo wget
%%capture
!pip install -q wget
!mkdir data

## 2.2 Obtenção dos Dados das Tabelas de Dimensão

Os comandos a seguir baixam os dados que povoam as tabelas de dimensão.

In [ ]:
#baixando os dados das tabelas de dimensão
import wget

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/data.csv"
wget.download(url, "data/data.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/funcionario.csv"
wget.download(url, "data/funcionario.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/equipe.csv"
wget.download(url, "data/equipe.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cargo.csv"
wget.download(url, "data/cargo.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cliente.csv"
wget.download(url, "data/cliente.csv")

'data/cliente (1).csv'

## 2.3 Obtenção dos Dados Tabelas de Fatos

Os comandos a seguir baixam os dados que povoam as tabelas de fatos.

In [ ]:
#baixando os dados das tabelas de fatos
url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/pagamento.csv"
wget.download(url, "data/pagamento.csv")

url = "https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/negociacao.csv"
wget.download(url, "data/negociacao.csv")

'data/negociacao (1).csv'

# 3 Apache Spark Cluster

## 3.1 Instalação

Neste *notebook* é criado um *cluster* Spark composto apenas por um **nó mestre**. Ou seja, o *cluster* não possui um ou mais **nós de trabalho** e o **gerenciador de cluster**. Nessa configuração, as tarefas (*tasks*) são realizadas no próprio *driver* localizado no **nó mestre**.

Para que o cluster possa ser criado, primeiramente é instalado o Java Runtime Environment (JRE) versão 8.

In [ ]:
#instalando Java Runtime Environment (JRE) versão 8
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Na sequência, é feito o *download* do Apache Spark versão 3.0.0.

In [ ]:
#baixando Apache Spark versão 3.0.0
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

Na sequência, são configuradas as variáveis de ambiente JAVA_HOME e SPARK_HOME. Isto permite que tanto o Java quanto o Spark possam ser encontrados.

In [ ]:
import os
#configurando a variável de ambiente JAVA_HOME
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#configurando a variável de ambiente SPARK_HOME
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

Por fim, são instalados dois pacotes da linguagem de programação Python, cujas funcionalidades são descritas a seguir.

> **Pacote findspark:** Usado para ler a variável de ambiente SPARK_HOME e armazenar seu valor na variável dinâmica de ambiente PYTHONPATH. Como resultado, Python pode encontrar a instalação do Spark.

> **Pacote pyspark:** PySpark é a API do Python para Spark. Ela possibilita o uso de Python, considerando que o *framework* Apache Spark encontra-se desenvolvido na linguagem de programação Scala.

In [ ]:
%%capture
#instalando o pacote findspark
!pip install -q findspark==1.4.2
#instalando o pacote pyspark
!pip install -q pyspark==3.0.0

## 3.2 Conexão

PySpark não é adicionado ao *sys.path* por padrão. Isso significa que não é possível importá-lo, pois o interpretador da linguagem Python não sabe onde encontrá-lo.

Para resolver esse aspecto, é necessário instalar o módulo `findspark`. Esse módulo mostra onde PySpark está localizado. Os comandos a seguir têm essa finalidade.


In [ ]:
#importando o módulo findspark
import findspark
#carregando a variávels SPARK_HOME na variável dinâmica PYTHONPATH
findspark.init()

Depois de configurados os pacotes e módulos e inicializadas as variáveis de ambiente, é possível iniciar o uso do Spark na aplicação de `data warehousing`. Para tanto, é necessário importar o comando `SparkSession` do módulo `pyspark.sql`. São utilizados os seguintes conceitos: <br>

- `SparkSession`: permite a criação de `DataFrames`. Como resultado, as tabelas relacionais podem ser manipuladas por meio de `DataFrames` e é possível realizar consultas OLAP por meio de comandos SQL. <br>
- `builder`: cria uma instância de SparkSession. <br>
- `appName`: define um nome para a aplicação, o qual pode ser visto na interface de usuário web do Spark. <br>
- `master`: define onde está o nó mestre do *cluster*. Como a aplicação é executada localmente e não em um *cluster*, indica-se isso pela *string* `local` seguida do parâmetro `[*]`. Ou seja, define-se que apenas núcleos locais são utilizados.
- `getOrCreate`: cria uma SparkSession. Caso ela já exista, retorna a instância existente.


**Observação**: A lista completa de todos os parâmetros que podem ser utilizados na inicialização do *cluster* pode ser encontrada neste [link](https://spark.apache.org/docs/latest/spark-standalone.html#cluster-launch-scripts).

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()

# 4 Geração dos DataFrames em Pandas da BI Solutions

Nesta seção são gerados os DataFrames em Pandas. Atenção aos nomes desses DataFrames.


In [ ]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
cargoPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cargo.csv')
clientePandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/cliente.csv')
dataPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/data.csv')
equipePandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/equipe.csv')
funcionarioPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/funcionario.csv')
negociacaoPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/negociacao.csv')
pagamentoPandas = pd.read_csv('https://raw.githubusercontent.com/GuiMuzziUSP/Data_Mart_BI_Solutions/main/pagamento.csv')

# 5 Geração dos DataFrames em Spark da BI Solutions

Nesta seção são gerados dos DataFrames em Spark. Atenção aos nomes desses DataFrames.


## 5.1 Criação dos DataFrames

In [ ]:
#criando os DataFrames em Spark
cargo = spark.read.csv(path="data/cargo.csv", header=True, sep=",")
cliente = spark.read.csv(path="data/cliente.csv", header=True, sep=",")
data = spark.read.csv(path="data/data.csv", header=True, sep=",")
equipe = spark.read.csv(path="data/equipe.csv", header=True, sep=",")
funcionario = spark.read.csv(path="data/funcionario.csv", header=True, sep=",")
negociacao = spark.read.csv(path="data/negociacao.csv", header=True, sep=",")
pagamento = spark.read.csv(path="data/pagamento.csv", header=True, sep=",")

## 5.2 Atualização dos Tipos de Dados

Nos comandos a seguir, primeiro são identificados quais colunas de quais `DataFrames` devem ser do tipo de dado inteiro. Na sequência, ocorre a conversão. Por fim, são exibidos os esquemas dos `DataFrames`, possibilitando visualizar a mudança de tipo de dados das colunas especificadas.

In [ ]:
# identificando quais colunas de quais DataFrames devem ser do tipo de dado inteiro
colunas_cargo = ["cargoPK"]
colunas_cliente = ["clientePK"]
colunas_data = ["dataPK", "dataDia", "dataMes", "dataBimestre", "dataTrimestre", "dataSemestre", "dataAno"]
colunas_equipe = ["equipePK"]
colunas_funcionario = ["funcPK", "funcDiaNascimento", "funcMesNascimento", "funcAnoNascimento"]
colunas_negociacao = ["equipePK", "clientePK", "dataPK", "quantidadeNegociacoes"]
colunas_pagamento = ["funcPK", "equipePK", "dataPK", "cargoPK", "quantidadeLancamentos"]

In [ ]:
# importando o tipo de dado desejado
from pyspark.sql.types import IntegerType


# atualizando o tipo de dado das colunas especificadas
# substituindo as colunas já existentes

for coluna in colunas_cargo:
  cargo = cargo.withColumn(coluna, cargo[coluna].cast(IntegerType()))

for coluna in colunas_cliente:
  cliente = cliente.withColumn(coluna, cliente[coluna].cast(IntegerType()))

for coluna in colunas_data:
  data = data.withColumn(coluna, data[coluna].cast(IntegerType()))

for coluna in colunas_equipe:
  equipe = equipe.withColumn(coluna, equipe[coluna].cast(IntegerType()))

for coluna in colunas_funcionario:
  funcionario = funcionario.withColumn(coluna, funcionario[coluna].cast(IntegerType()))

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(IntegerType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(IntegerType()))

Nos comandos a seguir, primeiro são identificados quais colunas de quais `DataFrames` devem ser do tipo de dado número de ponto flutuante. Na sequência, ocorre a conversão. Por fim, são exibidos os esquemas dos `DataFrames`, possibilitando visualizar a mudança de tipo de dados das colunas especificadas.

In [ ]:
# identificando quais colunas de quais DataFrames devem ser do tipo de dado número de ponto flutuante
colunas_negociacao = ["receita"]
colunas_pagamento = ["salario"]

In [ ]:
# importando o tipo de dado desejado
from pyspark.sql.types import FloatType


# atualizando o tipo de dado das colunas especificadas
# substituindo as colunas já existentes

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(FloatType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(FloatType()))

In [ ]:
# importando funções adicionais
from pyspark.sql.functions import round, desc

## 5.3 Criação de Visões Temporárias

In [ ]:
#criando as visões temporárias
cargo.createOrReplaceTempView("cargo")
cliente.createOrReplaceTempView("cliente")
data.createOrReplaceTempView("data")
equipe.createOrReplaceTempView("equipe")
funcionario.createOrReplaceTempView("funcionario")
negociacao.createOrReplaceTempView("negociacao")
pagamento.createOrReplaceTempView("pagamento")

NameError: ignored

# 6 Respostas

Lembre-se que é possível especificar as consultas analíticas usando Pandas, o método spark.sql() ou os métodos do módulo pyspark.sql.

## Especificação da Consulta Analítica da Questão 8

Considere a constelação de fatos da BI Solutions e a seguinte solicitação de consulta:

“Qual a quantidade de negociações por ano e pela cidade da equipe, considerando equipes localizadas na mesma cidade de seus clientes?” Devem ser exibidas as colunas na ordem e com os nomes especificados a seguir: ANO, CIDADE e TOTALNEGOCIACOES. Ordene as linhas exibidas primeiro por ano em ordem ascendente e depois por cidade em ordem ascendente.

Considerando a resposta para a consulta analítica, assinale a alternativa correta:



a.
As equipes localizadas na cidade de SAO PAULO possuem mais negociações do que as demais equipes em todos os anos.


b.
Quatro cidades distintas possuem equipes que negociaram mais de uma vez em 2018.


c.
Três cidades distintas possuem equipes que negociaram mais de uma vez em 2019.


d.
As equipes localizadas na cidade do RIO DE JANEIRO possuem menos negociações do que as demais equipes em todos os anos.


e.
Com exceção do ano de 2020, as negociações realizadas pelas equipes localizadas na cidade de SAO PAULO aumentaram paulatinamente.

In [ ]:
# Celula somente para testes para entendermos as bases

negociacao.show(4)
data.show(4)
equipe.show(4)
cliente.show(4)

query = """
select distinct quantidadeNegociacoes
from negociacao
"""

# Executa a query (consulta)
result_df = spark.sql(query)

# Mostra os resultados da consulta
print("Resultado utilizando o método spark.sql():")
result_df.show()

+--------+---------+------+--------+---------------------+
|equipePK|clientePK|dataPK| receita|quantidadeNegociacoes|
+--------+---------+------+--------+---------------------+
|       2|        9|    22|11564.75|                    1|
|       2|       24|    11| 17990.5|                    1|
|       2|       28|    21| 16335.9|                    1|
|       1|       30|    23| 8495.55|                    1|
+--------+---------+------+--------+---------------------+
only showing top 4 rows

+------+------------+-------+-------+------------+-------------+------------+-------+
|dataPK|dataCompleta|dataDia|dataMes|dataBimestre|dataTrimestre|dataSemestre|dataAno|
+------+------------+-------+-------+------------+-------------+------------+-------+
|     1|    1/1/2016|      1|      1|           1|            1|           1|   2016|
|     2|    2/1/2016|      2|      1|           1|            1|           1|   2016|
|     3|    3/1/2016|      3|      1|           1|            1|         

In [ ]:
# Resposta da Questão 8
# Não se esqueça de comentar detalhadamente a sua solução

query = """
select dataAno as ANO,
filialCidade as CIDADE,
count(quantidadeNegociacoes) as TOTALNEGOCIACOES
FROM negociacao as a
INNER JOIN data as b
ON a.dataPK = b.dataPK
INNER JOIN equipe as c
ON a.equipePK = c.equipePK
INNER JOIN cliente as d
ON a.clientePK = d.clientePK
where d.clienteCidade = c.filialCidade
group by ANO, CIDADE
order by ANO asc, CIDADE asc, TOTALNEGOCIACOES
"""

# Executa a query (consulta)
result_df = spark.sql(query)

# Mostra os resultados da consulta
print("Resultado utilizando o método spark.sql():")
result_df.show()


Resultado utilizando o método spark.sql():
+----+--------------+----------------+
| ANO|        CIDADE|TOTALNEGOCIACOES|
+----+--------------+----------------+
|2016|RIO DE JANEIRO|             130|
|2016|     SAO PAULO|             141|
|2017|  CAMPO GRANDE|              99|
|2017|RIO DE JANEIRO|             229|
|2017|     SAO PAULO|             209|
|2018|  CAMPO GRANDE|             198|
|2018|RIO DE JANEIRO|             339|
|2018|     SAO PAULO|             312|
|2019|  CAMPO GRANDE|             183|
|2019|        RECIFE|             175|
|2019|RIO DE JANEIRO|             353|
|2019|     SAO PAULO|             511|
|2020|  CAMPO GRANDE|             209|
|2020|        RECIFE|             146|
|2020|RIO DE JANEIRO|             324|
|2020|     SAO PAULO|             453|
+----+--------------+----------------+



In [ ]:
# .join(data, on="dataPK")n
# .join(equipe, on="equipePK")n
# .join(equipe, on="equipePK")n
# .where("dataPK BETWEEN 367 AND 731")n
# .select("equipeNome", "filialNome", "receita")n
# .groupBy("equipeNome", "filialNome")n
# .sum("receita")n
# .orderBy(desc("sum(receita)"))

In [ ]:
# '### Utilizando métodos do módulo pyspark.sql.
# Junção dos DataFrames
# joined_df = (
#     negociacao
#     .join(equipe, "equipePK")
#     .join(cliente, "clientePK")
#     .join(data, "dataPK")
#     .where(col("filialCidade") == col("clienteCidade"))
#     .select(col("dataAno").alias("ANO"), col("filialCidade").alias("CIDADE")) # Usar somente Letras maiúsculas em nome de colunas não é uma boa prática
# )


# Agrupando e fazendo a contagem do 'TOTALDENEGOCIACOES'
# result_df = (
#     joined_df
#     .groupBy("ANO", "CIDADE")
#     .count()
#     .withColumnRenamed("count", "TOTALNEGOCIACOES")
#     .orderBy("ANO", "CIDADE")
# )

# Mostra o resulta da consulta
# print("\nResultado utilizando métodos do módulo pyspark.sql:")
# result_df.show()'

## Especificação da Consulta Analítica da Questão 9

Considere a constelação de fatos da BI Solutions e a seguinte solicitação de consulta:

“Liste todas as agregações que podem ser geradas a partir da média dos salários dos funcionários que moram na região SUDESTE por sexo e por ano.” Arredonde a média dos salários para até duas casas decimais. Devem ser exibidas as colunas na ordem e com os nomes especificados a seguir: SEXO, ANO e MEDIASALARIO. Ordene as linhas exibidas primeiro por sexo, depois por ano, depois por média dos salários, todos em ordem ascendente.

Considerando a resposta para a consulta analítica, assinale a alternativa correta:


a.
São retornadas 10 linhas, das quais 5 linhas são referentes ao sexo feminino e 5 linhas são referentes ao sexo masculino.


b.
São retornadas 18 linhas, das quais 6 linhas são referentes ao sexo feminino e 6 linhas são referentes ao sexo masculino.


c.
A quarta e a quinta linhas retornadas contêm o mesmo valor de média dos salários e referem-se a anos diferentes.


d.
As médias dos salários das funcionárias de sexo feminino para os anos de 2017 e 2018 são menores do que as médias dos salários dos funcionários do sexo masculino para os anos de 2017 e 2018, respectivamente.


e.
São retornadas 13 linhas, das quais 6 linhas são referentes ao sexo feminino e 6 linhas são referentes ao sexo masculino.

In [ ]:
pagamento.show(4)
data.show(4)
funcionario.show(4)

+------+--------+------+-------+-------+---------------------+
|funcPK|equipePK|dataPK|cargoPK|salario|quantidadeLancamentos|
+------+--------+------+-------+-------+---------------------+
|   147|       2|     5|     64|1559.94|                    1|
|   124|       2|     5|    329|8102.77|                    1|
|   175|       1|     5|    328|2532.51|                    1|
|   171|       1|     5|    245| 7882.7|                    1|
+------+--------+------+-------+-------+---------------------+
only showing top 4 rows

+------+------------+-------+-------+------------+-------------+------------+-------+
|dataPK|dataCompleta|dataDia|dataMes|dataBimestre|dataTrimestre|dataSemestre|dataAno|
+------+------------+-------+-------+------------+-------------+------------+-------+
|     1|    1/1/2016|      1|      1|           1|            1|           1|   2016|
|     2|    2/1/2016|      2|      1|           1|            1|           1|   2016|
|     3|    3/1/2016|      3|      1|    

In [ ]:
# Resposta da Questão 9
# Não se esqueça de comentar detalhadamente a sua solução

query= '''SELECT
        func.funcSexo as SEXO,
        data.dataAno as ANO,
        round(avg(pag.salario),2) as MEDIASALARIO
        from pagamento pag
        INNER JOIN data ON pag.dataPK = data.dataPK
        inner join funcionario func on func.funcPK = pag.funcPK
        where func.funcRegiaoNome ='SUDESTE'
        group by CUBE (SEXO, ANO)
        order by SEXO asc, ANO asc, MEDIASALARIO asc
            '''

result_df = spark.sql(query)

result_df.show()

+----+----+------------+
|SEXO| ANO|MEDIASALARIO|
+----+----+------------+
|null|null|     7446.14|
|null|2016|     6542.64|
|null|2017|     7181.86|
|null|2018|     7582.23|
|null|2019|     7596.89|
|null|2020|     7596.89|
|   F|null|     7821.46|
|   F|2016|      8587.4|
|   F|2017|     8575.92|
|   F|2018|     8333.57|
|   F|2019|     7329.74|
|   F|2020|     7329.74|
|   M|null|     7322.23|
|   M|2016|     5903.65|
|   M|2017|     6763.65|
|   M|2018|      7345.4|
|   M|2019|     7689.92|
|   M|2020|     7689.92|
+----+----+------------+



In [ ]:
pagamento.join(funcionario, on="funcPK").join(data, on="dataPK")\
  .where("funcRegiaoNome = 'SUDESTE'")\
  .select("funcSexo", "dataAno", "salario")\
  .cube("funcSexo", "dataAno").avg("salario")\
  .withColumn("avg(salario)", round("avg(salario)", 2))\
  .orderBy("funcSexo", "dataAno")\
  .withColumnRenamed("funcSexo", "SEXO")\
  .withColumnRenamed("dataAno", "ANO")\
  .withColumnRenamed("avg(salario)", "MEDIASALARIO")\
  .show()

+----+----+------------+
|SEXO| ANO|MEDIASALARIO|
+----+----+------------+
|null|null|     7446.14|
|null|2016|     6542.64|
|null|2017|     7181.86|
|null|2018|     7582.23|
|null|2019|     7596.89|
|null|2020|     7596.89|
|   F|null|     7821.46|
|   F|2016|      8587.4|
|   F|2017|     8575.92|
|   F|2018|     8333.57|
|   F|2019|     7329.74|
|   F|2020|     7329.74|
|   M|null|     7322.23|
|   M|2016|     5903.65|
|   M|2017|     6763.65|
|   M|2018|      7345.4|
|   M|2019|     7689.92|
|   M|2020|     7689.92|
+----+----+------------+



## Especificação da Consulta Analítica da Questão 10

Considere a constelação de fatos da BI Solutions e a seguinte solicitação de consulta:

"Qual o lucro ou prejuízo médio de cada equipe localizada na região SUDESTE do BRASIL?", sendo que lucro representa a diferença entre a média das receitas e a média dos salários. Arredonde o lucro ou prejuízo para até duas casas decimais. Devem ser exibidas as colunas na ordem e com os nomes especificados a seguir: NOME DA EQUIPE, NOME DA FILIAL, CIDADE DA FILIAL, LUCRO OU PREJUÍZO.

Considerando a resposta para a consulta analítica, assinale a alternativa correta:


a.
As equipes localizadas na cidade de RIO DE JANEIRO garantem mais lucro à BI Solutions do que as equipes localizadas na cidade de SAO PAULO.


b.
Equipes que possuem WEB em seu nome, independentemente da filial e da cidade na qual estão localizadas, proveem mais lucro à BI Solutions do que as demais equipes.


c.
Os lucros gerados pelas equipes que possuem BI & ANALYTICS em seu nome, independentemente da filial e da cidade na qual estão localizadas, são maiores do que a soma dos lucros gerados pelas demais equipes.


d.
A soma dos lucros gerados pelas equipes que possuem APP - MOBILE em seu nome são maiores do que a soma dos lucros gerados pelas equipes que possuem APP - DESKTOP em seu nome, independentemente da filial e da cidade na qual estão localizadas.


e.
Todas as equipes tiveram lucro, independentemente da cidade na qual elas estão localizadas.

In [ ]:
# Resposta da Questão 10
# Não se esqueça de comentar detalhadamente a sua solução

# pagamento.printSchema()
pagamento.show(4)
negociacao.show(4)
equipe.show(4)

+------+--------+------+-------+-------+---------------------+
|funcPK|equipePK|dataPK|cargoPK|salario|quantidadeLancamentos|
+------+--------+------+-------+-------+---------------------+
|   147|       2|     5|     64|1559.94|                    1|
|   124|       2|     5|    329|8102.77|                    1|
|   175|       1|     5|    328|2532.51|                    1|
|   171|       1|     5|    245| 7882.7|                    1|
+------+--------+------+-------+-------+---------------------+
only showing top 4 rows

+--------+---------+------+--------+---------------------+
|equipePK|clientePK|dataPK| receita|quantidadeNegociacoes|
+--------+---------+------+--------+---------------------+
|       2|        9|    22|11564.75|                    1|
|       2|       24|    11| 17990.5|                    1|
|       2|       28|    21| 16335.9|                    1|
|       1|       30|    23| 8495.55|                    1|
+--------+---------+------+--------+---------------------+

In [ ]:
# Resposta da Questão 10
# Não se esqueça de comentar detalhadamente a sua solução

query= '''SELECT
        equ.equipeNome,
        avg(salario) as MEDIASALARIO,
        avg(receita) as MEDIARECEITA,
        avg(salario) - avg(receita) as MEDIA

        from equipe equ
        inner join negociacao neg on equ.equipePK = neg.equipePK
        inner join pagamento pag on equ.equipePK = pag.equipePK
        group by equ.equipeNome
        order by equ.equipeNome
            '''

result_df = spark.sql(query)

result_df.show()

+--------------+-----------------+------------------+
|    equipeNome|     MEDIASALARIO|      MEDIARECEITA|
+--------------+-----------------+------------------+
| APP - DESKTOP|7395.544118067273|17390.081895094652|
|  APP - MOBILE|8601.820626016066|14301.106680199791|
|BI & ANALYTICS|7578.408653202427| 64232.81567855812|
|           WEB|7380.769945513439| 8600.226078192974|
+--------------+-----------------+------------------+



In [ ]:
# Resposta da Questão 10
# Não se esqueça de comentar detalhadamente a sua solução

query= '''SELECT
        equ.equipeNome,
        avg(salario) as MEDIASALARIO,
        avg(receita) as MEDIARECEITA,
        avg(salario) - avg(receita) as MEDIA

        from equipe equ
        inner join (select equipePK, avg(rececita) as avg_receita from negociacao group by equipePK) neg on equ.equipePK = neg.equipePK
        inner join (select avg(pagamento) as avg_receita from negociacao) pag on equ.equipePK = pag.equipePK
        group by equ.equipeNome
        order by equ.equipeNome
            '''

result_df = spark.sql(query)

result_df.show()

In [ ]:
# Resposta da Questão 10
# Não se esqueça de comentar detalhadamente a sua solução

query= '''

with avg_salario as
        (SELECT
        equipePK,
        avg(salario) as MEDIASALARIO

        from pagamento pag
        group by equipePK),

avg_receita as
        (SELECT
        equipePK,
        avg(receita) as MEDIARECEITA

        from negociacao neg
        group by equipePK)

        select
        equ.equipeNome as `NOME DA EQUIPE`,
        equ.filialNome as `NOME DA FILIAL`,
        equ.filialCidade as `CIDADE DA FILIAL`,
        round(MEDIARECEITA - MEDIASALARIO, 2) as `LUCRO OU PREJUÍZO`
        from equipe as equ
        inner join avg_salario as sal on equ.equipePK = sal.equipePK
        inner join avg_receita as rec on equ.equipePK = rec.equipePK
        where equ.filialRegiaoNome = "SUDESTE"
        order by `NOME DA EQUIPE`, `NOME DA FILIAL`, `CIDADE DA FILIAL`, `LUCRO OU PREJUÍZO`

            '''

result_df = spark.sql(query)

result_df.show()

# NOME DA EQUIPE, NOME DA FILIAL, CIDADE DA FILIAL, LUCRO OU PREJUÍZO.

+--------------+--------------------+----------------+-----------------+
|NOME DA EQUIPE|      NOME DA FILIAL|CIDADE DA FILIAL|LUCRO OU PREJUÍZO|
+--------------+--------------------+----------------+-----------------+
|           WEB|RIO DE JANEIRO - ...|  RIO DE JANEIRO|          -370.04|
|  APP - MOBILE|RIO DE JANEIRO - ...|  RIO DE JANEIRO|          5828.01|
| APP - DESKTOP|RIO DE JANEIRO - ...|  RIO DE JANEIRO|         10452.61|
|  APP - MOBILE|SAO PAULO - AV. P...|       SAO PAULO|          5642.31|
|           WEB|SAO PAULO - AV. P...|       SAO PAULO|           680.37|
| APP - DESKTOP|SAO PAULO - AV. P...|       SAO PAULO|           9511.4|
|BI & ANALYTICS|SAO PAULO - AV. P...|       SAO PAULO|         55019.53|
+--------------+--------------------+----------------+-----------------+

